# Lab work #1
## Task 1

In [24]:
import numpy as np

def fill_array(row=4, col=4):
    return np.random.randint(50, size=(row,col))
    

def convolve(arr, row=4, col=4):
    kernel = np.matrix([[1, 0], [0, 1]])
    kernel_row = kernel_col = 2

    output_row = row - kernel_row + 1
    output_col = col - kernel_col + 1
    
    output = np.zeros((output_row, output_col), dtype="int")
    print(arr, "\n")
    for i in range(0, output_row):
        for j in range(0, output_col):
            output[i][j] = np.sum(np.multiply(arr[i:i+kernel_col,j:j+kernel_row], kernel))
    return output

arr = fill_array()
result = convolve(arr)
print(result)

[[28 15 26 36]
 [44 37  3 21]
 [35 21 23 20]
 [ 6 21 22  6]] 

65
18
47
65
60
23
56
43
29
[[65 18 47]
 [65 60 23]
 [56 43 29]]
